In [1]:
import datetime, pymysql
import time
import asyncio, aiohttp
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from bs4 import BeautifulSoup as BS

In [18]:
config = {
          'host':'localhost',
          'port':3306,
          'user':'root',
          'password':'ms020312',
          'db':'bili',
          'charset':'utf8mb4'
          }

In [3]:
def CT_video_danmus():
    
    global config
    
    connection = pymysql.connect(**config)

    try:
        with connection.cursor() as cursor:
            # Create a new record
            sql = """create table video_danmus(uid varchar(20), aid varchar(20), cid varchar(20),source_timestamp varchar(20), danmu_reptime int, 
            danmu_type varchar(20), danmu_size int, danmu_color varchar(20), danmu_creattime datetime, danmu_pole int, sender_ID varchar(20), rowID varchar(20),
            danmu_message varchar(300), record_time datetime)engine=innodb charset utf8mb4""" 
            cursor.execute(sql)

        connection.commit()
    finally:
        connection.close()

In [4]:
def CT_video_timestamps_done():
    
    global config
    
    connection = pymysql.connect(**config)

    try:
        with connection.cursor() as cursor:
            # Create a new record
            sql = """create table video_timestamps_done(uid varchar(20), aid varchar(20), cid varchar(20),source_timestamp varchar(20))engine=innodb charset utf8mb4""" 
            cursor.execute(sql)

        connection.commit()
    finally:
        connection.close()

In [5]:
async def danmu_xml_get(url):
    
    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(url=url) as resp:
                text = await resp.text()

                if resp.status != 200:
                    raise NameError

                return text

        except aiohttp.client_exceptions.ClientError:
            raise NameError

In [6]:
async def rolldate_get(cid):
    
    url = 'https://comment.bilibili.com/rolldate,%s' % cid
    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(url=url) as resp:
                text = await resp.text()

                if resp.status != 200:
                    raise NameError

                return text

        except aiohttp.client_exceptions.ClientError:
            raise NameError

In [7]:
async def video_danmus_coll(uid, aid, cid, source_timestamp, danmu_xml_url):
    global video_danmus_list
    global cidrowID_DF
    global timestamps_done_list
    
    text = await danmu_xml_get(danmu_xml_url)
    
    soup = BS(text, 'lxml')
    all_d = soup.select('d')
    now_str = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    try:
        rowIDs = cidrowID_DF.rowID[cidrowID_DF.cid == str(cid)].unique()
    except AttributeError:
        rowIDs = []
    
    collnum = 0
    
    for d in all_d:
        danmu_list = d['p'].split(',')
        
        rowID = danmu_list[7]
        if str(rowID) in rowIDs:
            continue
        
        danmu_reptime = danmu_list[0]
        danmu_type = danmu_list[1]
        danmu_size = danmu_list[2]
        danmu_color = danmu_list[3]
        danmu_creattime  = datetime.datetime.fromtimestamp(int(danmu_list[4])).strftime('%Y-%m-%d %H:%M:%S')
        danmu_pole = danmu_list[5]
        sender_ID = danmu_list[6] 
        rowID = danmu_list[7] 
        danmu_message = d.get_text()
        record_time = now_str
        
        temp = [uid,aid,cid,source_timestamp,danmu_reptime,danmu_type,danmu_size,danmu_color,danmu_creattime,danmu_pole,sender_ID,
                rowID,danmu_message,record_time]
        
        video_danmus_list.append(temp)
        collnum = collnum + 1
        
    print('uid:%s aid:%s url:%s been collocted: %s danmu' % (uid, aid, danmu_xml_url, collnum))
    
    temp2 = [uid, aid, cid, source_timestamp]
    timestamps_done_list.append(temp2)
    print('uid:%s aid:%s source_timestamp:%s url:%s been registered' % (uid, aid, source_timestamp, danmu_xml_url))

In [8]:
def get_cidrowID_DF(uid):
    global config
    
    connection = pymysql.connect(**config)

    cur = connection.cursor(pymysql.cursors.DictCursor)

    sql = "SELECT cid, rowID FROM video_danmus WHERE uid = %s"
    cur.execute(sql % uid)

    data = cur.fetchall()
    cur.close()
    connection.close()
    
    if len(data) == 0:
        DF = DataFrame()
        print('uid:%s has no cidrowID in Database' % uid)
        return DF

    DF = DataFrame(data)
    print('uid:%s has %s cidrowID in Database' % (uid,len(DF)))
    return DF

In [9]:
def get_cidtimestamp_DF(uid):
    global config
    
    connection = pymysql.connect(**config)

    cur = connection.cursor(pymysql.cursors.DictCursor)

    sql = "SELECT cid, source_timestamp FROM video_timestamps_done WHERE uid = %s GROUP BY cid, source_timestamp"
    cur.execute(sql % uid)

    data = cur.fetchall()
    cur.close()
    connection.close()
    
    if len(data) == 0:
        DF = DataFrame()
        print('uid:%s has no cidtimestamp in Database' % uid)
        return DF

    DF = DataFrame(data)
    print('uid:%s has %s cidtimestamp in Database' % (uid,len(DF)))
    return DF

In [10]:
def video_danmus_insert(video_danmus_list, timestamps_done_list):

    global config
    
    connection = pymysql.connect(**config)
    try:
        with connection.cursor() as cursor:

            sql = """INSERT INTO video_danmus(uid,aid,cid,source_timestamp,danmu_reptime,danmu_type,danmu_size,danmu_color,danmu_creattime,danmu_pole,sender_ID,
                rowID,danmu_message,record_time)VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"""

            try:
                cursor.executemany(sql,video_danmus_list)
                connection.commit()
                print('INSET %d ROWS in Table video_danmus' % len(video_danmus_list))
            except Exception as e:
                print(e)
                connection.rollback()
                raise Exception
            
            
            
            sql = """INSERT INTO video_timestamps_done(uid,aid,cid,source_timestamp)VALUES(%s,%s,%s,%s);"""
            
            try:
                cursor.executemany(sql,timestamps_done_list)
                connection.commit()
                print('INSET %d ROWS in Table video_timestamps_done' % len(timestamps_done_list))
            except Exception as e:
                print(e)
                connection.rollback()
                raise Exception
                
            
    finally:
        connection.close()
        
    
    
    

In [11]:
def get_cid_list(uid):
    global config
    
    connection = pymysql.connect(**config)

    cur = connection.cursor(pymysql.cursors.DictCursor)

    sql = "SELECT aid, cid FROM video_pages WHERE uid = %d"
    cur.execute(sql % uid)
    
    data = cur.fetchall()
    cidlist = []
    if len(data) == 0:
        print('uid: %s has no cid in Database' % uid)
        return cidlist

    DF = DataFrame(data)
    for i in range(len(DF)):
        cidlist.append([DF.iloc[i].values[0],DF.iloc[i].values[1]])

    print('uid: %s has %s cids in Database' % (uid, len(cidlist)))
    return cidlist

In [12]:
async def rolldate_coll(uid, aid, cid):
    global newdanmuxml_list_temp
    global cidtimestamp_DF
    
    try:
        timestamps = cidtimestamp_DF.source_timestamp[cidtimestamp_DF.cid == str(cid)].unique()
    except AttributeError:
        timestamps = []
    
    text = await rolldate_get(cid)
    
    if text == '':
        print('cid:%s has no rolldate' % cid)
        return
    data = eval(text)
    DF = DataFrame(data)
    collnum = 0
    for i in range(len(DF)):
        source_timestamp = DF.iloc[i].values[1]
        if str(source_timestamp) in timestamps:
            continue
        danmu_xml_url = 'https://comment.bilibili.com/dmroll,%s,%s' % (source_timestamp, cid)
        
        temp = [uid, aid, cid, source_timestamp, danmu_xml_url]        
        newdanmuxml_list_temp.append(temp)
        collnum = collnum + 1
        
    print('uid: %s aid: %s cid: %s has %s rolldate, been collected' % (uid, aid, cid, collnum))
    
    

In [13]:
def create_newdanmuxml_list(uid):
    
    global newdanmuxml_list
    global newdanmuxml_list_temp
    
    newdanmuxml_list = []
    
    cidlist = get_cid_list(uid)
    
    for [aid, cid] in cidlist:
        source_timestamp = 'current'
        danmu_xml_url = 'http://comment.bilibili.com/%s.xml' % (cid)
        temp = [uid, aid, cid, source_timestamp, danmu_xml_url]
        newdanmuxml_list.append(temp)
        
    print('uid: %s has %s current danmuxml_url' % (uid, len(newdanmuxml_list)))
        
    
    cidlist_num = len(cidlist)
    roundnum = (cidlist_num // 10) + 1
    
    st = 0
    
    while st < roundnum:
        start = st * 10
        end = start + 10
        newdanmuxml_list_temp = []
        
        asyncio.set_event_loop(asyncio.new_event_loop())
        loop = asyncio.get_event_loop()
        tasks = asyncio.gather(*[rolldate_coll(uid, aid, cid) for [aid, cid] in cidlist[start:end]])
        st = st + 1
        try:
            loop.run_until_complete(tasks)
        except NameError:
            print('too fast too fast')
            loop.close()
            time.sleep(60)
            st = st - 1
            continue
        
        newdanmuxml_list.extend(newdanmuxml_list_temp)
        loop.close()
            
    

In [14]:
def video_danmus_save(uid):
    global newdanmuxml_list
    global newdanmuxml_list_temp
    global video_danmus_list
    global timestamps_done_list
    global cidrowID_DF
    global cidtimestamp_DF

    newdanmuxml_list = []
    cidtimestamp_DF = get_cidtimestamp_DF(uid)
    create_newdanmuxml_list(uid)

    print('uid: %s has %s newdanmuxml to be saved'% (uid, len(newdanmuxml_list)))
    print('\r\n')

    newdanmuxml_list = newdanmuxml_list[1000:1200]
        
    newdanmuxml_num = len(newdanmuxml_list)
    roundnum = (newdanmuxml_num // 50) + 1

    st = 0

    while st < roundnum:
        st_inter = st + 10
        st_inter = min(st_inter, roundnum)
        video_danmus_list = []
        timestamps_done_list = []
        cidrowID_DF = get_cidrowID_DF(uid)
        while st < st_inter:
            start = st * 50
            end = start + 50

            asyncio.set_event_loop(asyncio.new_event_loop())
            loop = asyncio.get_event_loop()
            tasks = asyncio.gather(*[video_danmus_coll(uid, aid, cid, source_timestamp, danmu_xml_url) for
                                     [uid, aid, cid, source_timestamp, danmu_xml_url] in newdanmuxml_list[start:end]])

            st = st + 1
            try:
                loop.run_until_complete(tasks)
            except NameError:
                print('too fast too fast')
                loop.close()
                time.sleep(60)
                st = st - 1
                continue

            loop.close()

        orilistnum = len(video_danmus_list)
#        if orilistnum == 0:
#            continue

        video_danmus_list_DF = DataFrame(video_danmus_list)
        video_danmus_list_DFunique = video_danmus_list_DF.drop_duplicates(video_danmus_list_DF[[2,11]])
        arr = np.array(video_danmus_list_DFunique)
        video_danmus_list = arr.tolist()
        inslistnum = len(video_danmus_list)
        delnum = orilistnum - inslistnum
        print('orinum:%s insnum:%s delnum:%s' % (orilistnum, inslistnum, delnum))

        video_danmus_insert(video_danmus_list, timestamps_done_list)
        print('\r\n')
    

In [15]:
video_danmus_list = []
newdanmuxml_list = []
newdanmuxml_list_temp = []
timestamps_done_list = []
cidrowID_DF = None
cidtimestamp_DF = None

In [17]:
video_danmus_save(7349)

uid:7349 has 200 cidtimestamp in Database
uid: 7349 has 441 cids in Database
uid: 7349 has 441 current danmuxml_url
uid: 7349 aid: 20090588 cid: 32786326 has 27 rolldate, been collected
uid: 7349 aid: 20291549 cid: 33229541 has 21 rolldate, been collected
uid: 7349 aid: 20573687 cid: 33651286 has 15 rolldate, been collected
uid: 7349 aid: 20687491 cid: 33860119 has 13 rolldate, been collected
uid: 7349 aid: 20760130 cid: 33997207 has 11 rolldate, been collected
uid: 7349 aid: 20457214 cid: 33443800 has 18 rolldate, been collected
uid: 7349 aid: 20158445 cid: 32925441 has 25 rolldate, been collected
uid: 7349 aid: 20804061 cid: 34075715 has 10 rolldate, been collected
uid: 7349 aid: 20745013 cid: 33990733 has 11 rolldate, been collected
uid: 7349 aid: 20396426 cid: 33336165 has 20 rolldate, been collected
uid: 7349 aid: 19181551 cid: 31285371 has 47 rolldate, been collected
uid: 7349 aid: 19388944 cid: 31618411 has 38 rolldate, been collected
uid: 7349 aid: 19243962 cid: 31383337 has 46

uid: 7349 aid: 12632865 cid: 20780839 has 97 rolldate, been collected
uid: 7349 aid: 13077181 cid: 21465408 has 56 rolldate, been collected
uid: 7349 aid: 12800170 cid: 21040606 has 80 rolldate, been collected
uid: 7349 aid: 12970682 cid: 21302154 has 126 rolldate, been collected
uid: 7349 aid: 11980868 cid: 19772360 has 178 rolldate, been collected
uid: 7349 aid: 12542790 cid: 20638560 has 102 rolldate, been collected
uid: 7349 aid: 11648706 cid: 19245653 has 87 rolldate, been collected
uid: 7349 aid: 11563145 cid: 19110858 has 191 rolldate, been collected
uid: 7349 aid: 12002188 cid: 19805610 has 95 rolldate, been collected
uid: 7349 aid: 11404420 cid: 18855746 has 124 rolldate, been collected
uid: 7349 aid: 11855390 cid: 19575566 has 112 rolldate, been collected
uid: 7349 aid: 12213248 cid: 20128438 has 91 rolldate, been collected
uid: 7349 aid: 12069542 cid: 19911610 has 66 rolldate, been collected
uid: 7349 aid: 11763260 cid: 19426363 has 136 rolldate, been collected
uid: 7349 aid

uid: 7349 aid: 6216644 cid: 10096999 has 134 rolldate, been collected
uid: 7349 aid: 5889002 cid: 9562413 has 291 rolldate, been collected
uid: 7349 aid: 6289877 cid: 10219673 has 261 rolldate, been collected
uid: 7349 aid: 5862944 cid: 9520924 has 144 rolldate, been collected
uid: 7349 aid: 6089357 cid: 9885893 has 139 rolldate, been collected
uid: 7349 aid: 6006043 cid: 9750245 has 264 rolldate, been collected
uid: 7349 aid: 5678701 cid: 9221412 has 139 rolldate, been collected
uid: 7349 aid: 6069275 cid: 9852808 has 176 rolldate, been collected
uid: 7349 aid: 6118687 cid: 9934582 has 364 rolldate, been collected
uid: 7349 aid: 5953062 cid: 9665911 has 198 rolldate, been collected
uid: 7349 aid: 6167310 cid: 10015848 has 215 rolldate, been collected
uid: 7349 aid: 5552121 cid: 9017737 has 254 rolldate, been collected
uid: 7349 aid: 5849814 cid: 9499333 has 149 rolldate, been collected
uid: 7349 aid: 5723941 cid: 9294808 has 220 rolldate, been collected
uid: 7349 aid: 5651738 cid: 917

uid: 7349 aid: 3063686 cid: 4810548 has 516 rolldate, been collected
uid: 7349 aid: 3060717 cid: 4805063 has 314 rolldate, been collected
uid: 7349 aid: 2954229 cid: 4625458 has 230 rolldate, been collected
uid: 7349 aid: 3084921 cid: 4846086 has 255 rolldate, been collected
uid: 7349 aid: 3322965 cid: 5254434 has 448 rolldate, been collected
uid: 7349 aid: 3111702 cid: 4892425 has 247 rolldate, been collected
uid: 7349 aid: 2959200 cid: 4633593 has 316 rolldate, been collected
uid: 7349 aid: 2891575 cid: 4518558 has 100 rolldate, been collected
uid: 7349 aid: 3038549 cid: 4768146 has 302 rolldate, been collected
uid: 7349 aid: 2918738 cid: 4565937 has 321 rolldate, been collected
uid: 7349 aid: 2979764 cid: 4668444 has 236 rolldate, been collected
uid: 7349 aid: 3105426 cid: 4882139 has 521 rolldate, been collected
uid: 7349 aid: 3004002 cid: 4709448 has 353 rolldate, been collected
uid: 7349 aid: 2923806 cid: 4574488 has 262 rolldate, been collected
uid: 7349 aid: 2935018 cid: 459203

KeyboardInterrupt: 

In [18]:
video_danmus_list

[[7349,
  '2891575',
  '4518558',
  'current',
  '60.32400',
  '1',
  '25',
  '16777215',
  '2015-09-11 11:11:13',
  '0',
  '5d2b8647',
  '1202019653',
  '第一',
  '2018-03-26 12:01:43'],
 [7349,
  '2891575',
  '4518558',
  'current',
  '12.19300',
  '1',
  '25',
  '16777215',
  '2015-09-11 11:11:13',
  '0',
  '5c600f7f',
  '1202019717',
  '第5。。。',
  '2018-03-26 12:01:43'],
 [7349,
  '2891575',
  '4518558',
  'current',
  '71.63300',
  '1',
  '25',
  '16777215',
  '2015-09-11 11:11:21',
  '0',
  '9e9080e0',
  '1202028221',
  '第六',
  '2018-03-26 12:01:43'],
 [7349,
  '2891575',
  '4518558',
  'current',
  '0.56700',
  '1',
  '25',
  '16777215',
  '2015-09-11 11:11:21',
  '0',
  'a7b20cb9',
  '1202028549',
  '138',
  '2018-03-26 12:01:43'],
 [7349,
  '2891575',
  '4518558',
  'current',
  '10.82700',
  '1',
  '25',
  '16777215',
  '2015-09-11 11:11:25',
  '0',
  '140c3f86',
  '1202047039',
  '哈哈，前十',
  '2018-03-26 12:01:43'],
 [7349,
  '2891575',
  '4518558',
  'current',
  '133.84800',
  

In [19]:
dfdanmu = DataFrame(video_danmus_list)

In [32]:
dfdanmu

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,7349,2891575,4518558,current,60.32400,1,25,16777215,2015-09-11 11:11:13,0,5d2b8647,1202019653,第一,2018-03-26 12:01:43
1,7349,2891575,4518558,current,12.19300,1,25,16777215,2015-09-11 11:11:13,0,5c600f7f,1202019717,第5。。。,2018-03-26 12:01:43
2,7349,2891575,4518558,current,71.63300,1,25,16777215,2015-09-11 11:11:21,0,9e9080e0,1202028221,第六,2018-03-26 12:01:43
3,7349,2891575,4518558,current,0.56700,1,25,16777215,2015-09-11 11:11:21,0,a7b20cb9,1202028549,138,2018-03-26 12:01:43
4,7349,2891575,4518558,current,10.82700,1,25,16777215,2015-09-11 11:11:25,0,140c3f86,1202047039,哈哈，前十,2018-03-26 12:01:43
5,7349,2891575,4518558,current,133.84800,1,25,16777215,2015-09-11 11:11:26,0,b8258a16,1202047533,前十！！,2018-03-26 12:01:43
6,7349,2891575,4518558,current,2.31500,1,25,15138834,2015-09-11 11:11:29,0,ee24c596,1202057361,好短,2018-03-26 12:01:43
7,7349,2891575,4518558,current,24.65000,1,25,16777215,2015-09-11 11:11:30,0,7cc2d64e,1202059339,以后是不是能出张专辑?,2018-03-26 12:01:43
8,7349,2891575,4518558,current,142.64200,1,25,16777215,2015-09-11 11:11:35,0,e19ad03,1202064091,前好多,2018-03-26 12:01:43
9,7349,2891575,4518558,current,150.05800,1,25,16777215,2015-09-11 11:11:36,0,18d6010,1202065227,啦啦啦~,2018-03-26 12:01:43
